In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import datetime
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn

from torch import optim
from torch.utils.data import Dataset, DataLoader
import sys 
sys.path.append("..") 
from model.dataset import *
from model.GaussianDiffusion_new1228 import GaussianDiffusion #可以自己调

from metrics_calculate import *

In [2]:
device = 'cuda:0'
model_path = "./DiffusionSpinalMRISynthesis/model_save/pix2pix_baseline_500epoch.pth"
net_G = torch.load(model_path, map_location=device)['net_G']
net_G.eval()
print("模型已加载")

模型已加载


需要定量描述，把所有测试集的图片测试一遍，计算指标！

In [3]:
spinal_test_dir = "./DiffusionSpinalMRISynthesis/Data_MRI/test_spinal_MRI"
test_spinal_dataset = MRI_patient_Dataset_fortest(dir_path=spinal_test_dir)
test_spinal_dataloader = DataLoader(test_spinal_dataset, batch_size=64, shuffle=False, num_workers=16, pin_memory=True)
def get_data_from_batch(batch_data: torch.Tensor, device= device):
    return (batch_data[:,:3,:,:].to(device), batch_data[:,3:,:,:].cpu().detach())

def Test_model_metrics(net_G, test_dataloader):
    df_metrics = {'AUC':[],'MSE':[], 'SSIM':[], 'PSNR':[]} #
    # df_metrics = []
    net_G.eval()
    for idx, batch_data in enumerate(tqdm(test_dataloader), 1):
        input_img, target_img = get_data_from_batch(batch_data)
        with torch.no_grad():
            fake_img = net_G(input_img.to(device)).squeeze(dim=1).detach().cpu().numpy()
        # 在-1+1 maxminnorm计算全局指标
        # print(fake_img.shape)
        AUC, MSE, SSIM, PSNR= cal_metric_list(fake_img, target_img.squeeze(dim=1).numpy(), method='8bit', norm=True)
        df_metrics['AUC'].extend(AUC)
        df_metrics['MSE'].extend(MSE)
        df_metrics['SSIM'].extend(SSIM)
        df_metrics['PSNR'].extend(PSNR)
        # df_metrics['NRMSE'].extend(NRMSE)
        
    df_metrics = pd.DataFrame(df_metrics)
    print(df_metrics.mean())

    return df_metrics

In [ ]:
metrics_df = Test_model_metrics(net_G, test_spinal_dataloader)
metrics_df

In [ ]:
metrics_df

In [ ]:
metrics_df = pd.read_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/pix2pix_1231_baseline_test.csv").drop("Unnamed: 0", axis=1)
df_metrics_sum = pd.concat([metrics_df.mean(axis=0), metrics_df.std(axis=0)], axis=1).rename(columns={0:'metrics_mean', 1:'metrics_std'})
df_metrics_sum.to_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/pix2pix_test_sum.csv")
df_metrics_sum

In [7]:
metrics_df.to_csv("./DiffusionSpinalMRISynthesis/Test_models/metrics/pix2pix_baseline_test.csv")

In [ ]:
device = 'cuda:0'
model_path = "./DiffusionSpinalMRISynthesis/model_save/pix2pix_baseline_500epoch.pth"
net_G = torch.load(model_path, map_location=device)['model']
net_G.eval()
# 这样可以固定随机数种子
net_G = GaussianDiffusion(
    image_size=256,
    model = net_G.model,
    timesteps = 1000,           # number of steps
    sampling_timesteps = 16,    # using ddim for faster inference 
    objective = 'pred_v', #pred_v pred_x0崩 pred_noise

).to(device)

spinal_test_dir = "./DiffusionSpinalMRISynthesis/Data_MRI/test_spinal_MRI"
output_path = "./DiffusionSpinalMRISynthesis/results_output_pictures/pix2pix"

def Test_model_onepatient(net_G:GaussianDiffusion, patient_id, output_path=output_path):
    os.makedirs(output_path, exist_ok=True)
    fnames = glob(os.path.join(spinal_test_dir, '*'+patient_id+'*'))
    test_onepatient_dataset = MRI_patient_Dataset_fortestpatient(dir_path=spinal_test_dir, patient_id=patient_id)
    net_G.eval()
    input_img = torch.concat([test_onepatient_dataset[i][[0,1,2],:,:].unsqueeze(dim=0) for i in range(len(test_onepatient_dataset))],dim=0)
    target_img = torch.concat([test_onepatient_dataset[i][3:,:,:].unsqueeze(dim=0) for i in range(len(test_onepatient_dataset))],dim=0)
    with torch.no_grad():
        fake_img = net_G.ddim_sample(shape =target_img.shape, source_img = input_img.to(device), 
            sampling_timesteps = 16, random_seed=666).detach().cpu().squeeze(dim=1).numpy()
    
    fake_img_norm = tanhnorm_layerHW(fake_img)
    patient_save_path = os.path.join(output_path, patient_id)
    os.makedirs(patient_save_path, exist_ok=True)

    for i in range(fake_img.shape[0]):
        img = Image.fromarray(fake_img_norm[i,:,:])
        i_savepath = os.path.join(patient_save_path, patient_id+'_T1CE_pred_'+fnames[i].split('_')[-1])
        img.save(i_savepath)



In [ ]:
patient_id_list = list(set([path.split('_')[0]+'_'+path.split('_')[1]+'_'+path.split('_')[2] for path in os.listdir(spinal_test_dir)]))
for patient_id in tqdm(patient_id_list):
    Test_model_onepatient(net_G = net_G, patient_id = patient_id)